# Extrator Clipping
Objetivo: A planilha informada ao BNDES com os artigos de imprensa continha somente o link da publicação, foi necessário navegar pelos links e baixar conteúdo da postagem.

In [ ]:
!apt-get update 
!apt install chromium-chromedriver

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 4.2 MB/s 
     |████████████████████████████████| 138 kB 40.0 MB/s 
     |████████████████████████████████| 358 kB 36.5 MB/s 
     |████████████████████████████████| 4.0 MB 40.6 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.1 MB/s 


In [ ]:
import requests
from requests.models import Response
import re
from tqdm import tqdm
from tqdm.auto import trange
import time
import os

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from dataclasses import dataclass
from typing import List, Dict, Union

from bs4 import BeautifulSoup
from bs4.element import Tag
import unidecode
from datetime import datetime
import time

import pandas as pd
from pandas.core.series import Series
import json

import numpy as np

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
chrome_options.add_argument('--ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
GDRIVE_PATH:str = '/content/drive/MyDrive'
DATASET_ROOT_PATH: str = os.path.join(GDRIVE_PATH, 'dataset', 'projeto-final')
DATASET_IMPRENSA: str = os.path.join(DATASET_ROOT_PATH, 'imprensa', 'original')
DATASET_MENCOES: str = os.path.join(DATASET_ROOT_PATH, 'mencoes')
ARQUIVOS_IMPRENSA_PROCESSAMENTO_SIMPLES: List[str] = ['2020-06-Imprensa.xlsx', '2020-07-Imprensa.xlsx', '2020-08-Imprensa.xlsx', '2020-09-Imprensa.xlsx', 
                                                      #'2020-10-Imprensa.xlsx', '2020-11-Imprensa.xlsx', 
                                                      #'2020-12-Imprensa.xlsx'
                                                      ]
ARQUIVO_IMPRENSA_PROCESSAMENTO_CONSOLIDADO: str = '2021-2022-Imprensa_Consolidada.xlsx'

In [ ]:
def carrega_arquivos_imprensa_simples() -> None:
    vet_artigos: np.ndarray

    for arq in ARQUIVOS_IMPRENSA_PROCESSAMENTO_SIMPLES:
        df =  pd.read_excel(os.path.join(DATASET_IMPRENSA, arq))
        df['texto_artigo'] = pd.Series(dtype='str')
        cont_linha: int = 0
        vet_artigos = df[['ID', 'Data','Link para o sistema']].to_numpy()
        
        for row in tqdm(vet_artigos, desc=f'Carregando {arq}'):
            try:
                wd.get(row[2])
                time.sleep(5)
                element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '#nNoticia p'))
                WebDriverWait(wd, 15).until(element_present)

                soup = BeautifulSoup(wd.page_source, 'lxml')
                tag_conteudo: Tag = soup.find(id="nNoticia")
                conteudo_artigo: str = ''
                for tag_paragrafo in tag_conteudo.find_all('p'):        
                    if tag_paragrafo.contents and len(tag_paragrafo.contents) > 0:
                        conteudo_paragrafo: str = " ".join(str(item) for item in tag_paragrafo.contents).strip()
                        conteudo_paragrafo = unidecode.unidecode(conteudo_paragrafo)
                        conteudo_artigo += re.sub("<[^>]*>", "", conteudo_paragrafo)
                
                df.iloc[cont_linha, -1] = conteudo_artigo
            except Exception as e:
                print(f'Falha no carregamento da página {row[2]} ', e, '\n')
                df.iloc[cont_linha, -1] = 'CONTEUDO_INDISPONIVEL'
            finally:
                cont_linha += 1

        df.to_csv(arq.replace(".xlsx",".csv"), sep="|")
                


In [ ]:
carrega_arquivos_imprensa_simples()

Carregando 2020-09-Imprensa.xlsx: 100%|██████████| 210/210 [28:46<00:00,  8.22s/it]


In [ ]:
def carrega_arquivo_imprensa_consolidado() -> None:
    vet_artigos: np.ndarray
    tam_lote: int = 20

    df =  pd.read_excel(os.path.join(DATASET_IMPRENSA,  ARQUIVO_IMPRENSA_PROCESSAMENTO_CONSOLIDADO))
    df['Data da publicação'] = df['Data da publicação'].str.extract(r"(\d\d/\d\d/[\d]*)")
    df['texto_artigo'] = pd.Series(dtype='str')
    cont_linha: int = 0
    vet_artigos = df[['Matéria', 'Data da publicação','Link web - Texto']].to_numpy()
    #vet_artigos = vet_artigos[0:7]
    
    for row in tqdm(vet_artigos, desc=f'Carregando Impensa Consolidado'):
        try:
            wd.get(row[2])
            time.sleep(5)
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '#nNoticia p'))
            WebDriverWait(wd, 15).until(element_present)

            soup = BeautifulSoup(wd.page_source, 'lxml')
            tag_conteudo: Tag = soup.find(id="nNoticia")
            conteudo_artigo: str = ''
            for tag_paragrafo in tag_conteudo.find_all('p'):        
                if tag_paragrafo.contents and len(tag_paragrafo.contents) > 0:
                    conteudo_paragrafo: str = " ".join(str(item) for item in tag_paragrafo.contents).strip()
                    conteudo_paragrafo = unidecode.unidecode(conteudo_paragrafo)
                    conteudo_artigo += re.sub("<[^>]*>", "", conteudo_paragrafo)
            
            df.iloc[cont_linha, -1] = conteudo_artigo
        except Exception as e:
            print(f'Falha no carregamento da página {row[2]} - {str(e)} \n')
            df.iloc[cont_linha, -1] = 'CONTEUDO_INDISPONIVEL'
        finally:
            cont_linha += 1
            if cont_linha % tam_lote == 0:
                df.to_csv("consolidado.csv", sep="|")            

    df.to_csv("consolidado.csv", sep="|")

In [ ]:
carrega_arquivo_imprensa_consolidado()